# <font color=blue>Assignments for "Feature Engineering - Part 2"</font>

In this assignment, you are going to use a dataset related to the US education system. Please download the ([dataset](https://www.kaggle.com/noriuk/us-education-datasets-unification-project/home)) from Kaggle. You are going to use `states_all.csv` within this dataset.

To complete this assignment, submit the Github link of the Jupyter notebook file containing solutions to the questions below. You can talk to your mentor on your head or ask Slack at office time.


In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.stats.mstats import winsorize
from statsmodels.formula.api import ols
import statsmodels.api as sm
from sqlalchemy import create_engine
from scipy.stats import jarque_bera
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings

In [2]:
states = pd.read_csv("../../data/EDA/states_all.csv",parse_dates=['YEAR'])

In [3]:
var_null=states.isnull().sum()
var_missing=var_null[var_null!=0].index

states_median_imputation=states.copy()

for i in range(0,(len(var_missing))): 
    states_median_imputation[var_missing[i]].fillna(states_median_imputation[var_missing[i]].median(),inplace=True)

**(1)** Create a variable that contains the weighted average of the grades in the dataset. The number of students in the fourth grade is different from that of the eighth grade. So you will need a weighted average!

In [4]:
states_median_imputation["mean_score"] = (states_median_imputation["GRADES_4_G"]* 
                                 ((states_median_imputation["AVG_MATH_4_SCORE"] + states_median_imputation["AVG_READING_4_SCORE"]) * 0.5) + 
                                 states_median_imputation["GRADES_8_G"]
                                 * ((states_median_imputation["AVG_MATH_8_SCORE"] + 
                                     states_median_imputation["AVG_READING_8_SCORE"]) * 
                                    0.5))/ (states_median_imputation["GRADES_4_G"] + states_median_imputation["GRADES_8_G"])

**(2)** What is the correlation between the variable you just created and the types of expenditures? Which expenditure item has more correlation than others?

In [5]:
states_median_imputation[["mean_score", "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE",
              "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]].corr()

,mean_score,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
mean_score,1.000000,0.039933,0.047468,0.043488,-0.000882,-0.010094
TOTAL_EXPENDITURE,0.039933,1.000000,0.991571,0.993475,0.943541,0.933904
INSTRUCTION_EXPENDITURE,0.047468,0.991571,1.000000,0.977511,0.907780,0.898410
SUPPORT_SERVICES_EXPENDITURE,0.043488,0.993475,0.977511,1.000000,0.949709,0.919539
OTHER_EXPENDITURE,-0.000882,0.943541,0.907780,0.949709,1.000000,0.921472
CAPITAL_OUTLAY_EXPENDITURE,-0.010094,0.933904,0.898410,0.919539,0.921472,1.000000


The most correlated variable with the overall score is the instruction expenditure.

**(3)** Now apply the Principal Components Analysis (PCA) for the four expenditure items! How much of the total variance can be explained by the first component?

In [6]:
X = states_median_imputation[["INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
                  "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]]

X = StandardScaler().fit_transform(X)
pca = PCA()
principalComponents=pca.fit_transform(X)

In [7]:
exp_var= pca.explained_variance_ratio_
cumsum_var=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
print(exp_var)
print(cumsum_var)

print('The percentage of total variance in the dataset explained by the first component is', exp_var[0])

[0.94690205 0.02999931 0.01953914 0.0035595 ]
[ 94.69  97.69  99.64 100.  ]
The percentage of total variance in the dataset explained by the first component is 0.9469020470223894


**(4)** What is the correlation between the GPA you created and the first principal component?

In [8]:
states_median_imputation['PC1']=principalComponents[:,0]

states_median_imputation[["mean_score","PC1"]].corr()

,mean_score,PC1
mean_score,1.00000,0.02075
PC1,0.02075,1.00000


**(5)** When you need to choose the most appropriate variables for your model, would you prefer the first basic variables instead of the expenditure items? Why?

In [9]:
states_median_imputation[["mean_score","PC1","TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE",
              "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]].corr()

,mean_score,PC1,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
mean_score,1.000000,0.020750,0.039933,0.047468,0.043488,-0.000882,-0.010094
PC1,0.020750,1.000000,0.992502,0.972333,0.988534,0.970889,0.960392
TOTAL_EXPENDITURE,0.039933,0.992502,1.000000,0.991571,0.993475,0.943541,0.933904
INSTRUCTION_EXPENDITURE,0.047468,0.972333,0.991571,1.000000,0.977511,0.907780,0.898410
SUPPORT_SERVICES_EXPENDITURE,0.043488,0.988534,0.993475,0.977511,1.000000,0.949709,0.919539
OTHER_EXPENDITURE,-0.000882,0.970889,0.943541,0.907780,0.949709,1.000000,0.921472
CAPITAL_OUTLAY_EXPENDITURE,-0.010094,0.960392,0.933904,0.898410,0.919539,0.921472,1.000000


The correlation between the average score and the Instruction Expenditure is still the highest one among others. Therefore I will chooese this variable for my model.